# **S-Box**

In [ ]:
import numpy as np

In [ ]:
sbox = [
    99, 205, 85, 71, 25, 127, 113, 219, 63, 244, 109, 159, 11, 228, 94, 214,
    77, 177, 201, 78, 5, 48, 29, 30, 87, 96, 193, 80, 156, 200, 216, 86,
    116, 143, 10, 14, 54, 169, 148, 68, 49, 75, 171, 157, 92, 114, 188, 194,
    121, 220, 131, 210, 83, 135, 250, 149, 253, 72, 182, 33, 190, 141, 249, 82,
    232, 50, 21, 84, 215, 242, 180, 198, 168, 167, 103, 122, 152, 162, 145, 184,
    43, 237, 119, 183, 7, 12, 125, 55, 252, 206, 235, 160, 140, 133, 179, 192,
    110, 176, 221, 134, 19, 6, 187, 59, 26, 129, 112, 73, 175, 45, 24, 218,
    44, 66, 151, 32, 137, 31, 35, 147, 236, 247, 117, 132, 79, 136, 154, 105,
    199, 101, 203, 52, 57, 4, 153, 197, 88, 76, 202, 174, 233, 62, 208, 91,
    231, 53, 1, 124, 0, 28, 142, 170, 158, 51, 226, 65, 123, 186, 239, 246,
    38, 56, 36, 108, 8, 126, 9, 189, 81, 234, 212, 224, 13, 3, 40, 64,
    172, 74, 181, 118, 39, 227, 130, 89, 245, 166, 16, 61, 106, 196, 211, 107,
    229, 195, 138, 18, 93, 207, 240, 95, 58, 255, 209, 217, 15, 111, 46, 173,
    223, 42, 115, 238, 139, 243, 23, 98, 100, 178, 37, 97, 191, 213, 222, 155,
    165, 2, 146, 204, 120, 241, 163, 128, 22, 90, 60, 185, 67, 34, 27, 248,
    164, 69, 41, 230, 104, 47, 144, 251, 20, 17, 150, 225, 254, 161, 102, 70
]

# **NL**

In [ ]:
def walsh_hadamard_transform(boolean_function):
    n = boolean_function.size
    transformed = 1 - 2 * boolean_function

    for i in range(int(np.log2(n))):
        step = 2**i
        for j in range(0, n, 2*step):
            for k in range(step):
                a = transformed[j + k]
                b = transformed[j + k + step]
                transformed[j + k] = a + b
                transformed[j + k + step] = a - b

    return transformed

def calculate_nonlinearity(sbox):
    n = int(np.log2(len(sbox)))
    m = n

    sbox = np.array([list(map(int, f"{val:0{m}b}")) for val in sbox])

    nonlinearity = float('inf')
    for i in range(m):
        boolean_function = sbox[:, i]

        walsh_transform = walsh_hadamard_transform(boolean_function)

        walsh_str = ', '.join(map(str, walsh_transform))
        print(f"Walsh-Hadamard transform untuk output ke-{i+1}: [{walsh_str}]")

        max_correlation = np.max(np.abs(walsh_transform))

        current_nonlinearity = 2**(n-1) - (max_correlation // 2)
        print(f"Nonlinearity untuk output ke-{i+1}: {current_nonlinearity}")

        nonlinearity = min(nonlinearity, current_nonlinearity)

    return float(nonlinearity)

nl = calculate_nonlinearity(sbox)
print(f"\n--- Hasil Akhir ---")
print("Nonlinearity dari S-Box:", nl)

Walsh-Hadamard transform untuk output ke-1: [0, 8, 24, 24, 8, -8, -16, -8, 24, -8, 8, 0, -8, -16, -24, -8, 16, 24, 24, -8, -8, 8, -16, 24, -24, -24, -24, 32, 24, -16, -8, -24, -12, -4, -20, -4, 12, -4, -12, 12, 28, -4, -20, -12, -20, 4, -4, -4, -20, -12, -12, 4, 4, -12, -4, -12, -12, -12, 20, -4, 20, -20, -12, 20, 24, -16, 16, 32, 0, -16, -8, 0, 8, 24, -8, 0, 8, 0, -24, 24, 16, 24, 8, -24, -8, 24, 16, 8, 32, 0, -16, -24, 16, -8, -16, 16, 20, -4, 28, -20, -4, -4, 4, 12, -12, -12, 20, -4, -12, -4, -28, 20, -12, 12, -4, -4, -4, 12, 4, -4, -12, 4, 20, 12, -28, -4, 20, -12, -8, 16, -16, 16, 8, 24, 0, 24, 24, 24, 8, 16, -16, 24, 16, 32, 0, 8, -24, -8, 32, 0, -24, -16, 0, -16, 0, -8, -8, 16, 8, -24, 12, 20, -12, -12, -4, -4, 4, -4, 12, -4, 12, -12, 4, -4, -12, 4, -4, 20, -12, -28, 28, -20, -12, -4, -4, -4, 12, -28, 20, -4, 20, 20, 0, 24, 8, 8, 0, 0, 24, 0, -24, 8, -24, 16, -16, 8, -16, -16, 0, 24, 8, 8, -16, -16, -24, -16, 8, -24, -24, 16, 16, 8, 16, 16, 28, 20, -28, 20, -20, 12, 4, 28, 4, -2

# **SAC**

In [ ]:
def hamming_weight(x):
    return bin(x).count('1')

def calculate_sac(sbox, n):
    print("\n=== Menghitung SAC ===")
    total_weight = 0
    total_cases = 0

    for i in range(2**n):
        original_output = sbox[i]
        print(f"\nInput: {i:08b} -> Output: {original_output:08b}")

        for bit in range(n):
            flipped_input = i ^ (1 << bit)
            flipped_output = sbox[flipped_input]
            diff = original_output ^ flipped_output

            weight = hamming_weight(diff)
            total_weight += weight
            total_cases += n

            print(f"  Flip bit {bit}:")
            print(f"    Input flipped: {flipped_input:08b} -> Output: {flipped_output:08b}")
            print(f"    XOR: {diff:08b} (Hamming weight: {weight})")

    sac_value = total_weight / total_cases
    print(f"\nSAC Value: {sac_value:.5f}")
    print("Ideal SAC adalah 0.5, artinya rata-rata perubahan 50% bit output akibat perubahan 1 bit input.")
    return sac_value


if len(sbox) != 256:
    print("Error: S-box harus memiliki panjang 256 elemen.")
else:
    n = 8
    final_sac = calculate_sac(sbox, n)
    print(f"\nHasil SAC Akhir: {final_sac:.5f}")


Output streaming akan dipotong hingga 5000 baris terakhir.
  Flip bit 7:
    Input flipped: 10111111 -> Output: 01101011
    XOR: 00111001 (Hamming weight: 4)

Input: 01000000 -> Output: 11101000
  Flip bit 0:
    Input flipped: 01000001 -> Output: 00110010
    XOR: 11011010 (Hamming weight: 5)
  Flip bit 1:
    Input flipped: 01000010 -> Output: 00010101
    XOR: 11111101 (Hamming weight: 7)
  Flip bit 2:
    Input flipped: 01000100 -> Output: 11010111
    XOR: 00111111 (Hamming weight: 6)
  Flip bit 3:
    Input flipped: 01001000 -> Output: 10101000
    XOR: 01000000 (Hamming weight: 1)
  Flip bit 4:
    Input flipped: 01010000 -> Output: 00101011
    XOR: 11000011 (Hamming weight: 4)
  Flip bit 5:
    Input flipped: 01100000 -> Output: 01101110
    XOR: 10000110 (Hamming weight: 3)
  Flip bit 6:
    Input flipped: 00000000 -> Output: 01100011
    XOR: 10001011 (Hamming weight: 4)
  Flip bit 7:
    Input flipped: 11000000 -> Output: 11100101
    XOR: 00001101 (Hamming weight: 3)

Inp

# **BIC-NL**

In [ ]:
def calculate_bic_nl(sbox_values):
    sbox_size = len(sbox_values)
    bit_size = 8
    total_bit_pairs = bit_size * (bit_size - 1) / 2
    total_nonlinearity_score = 0

    steps = []


    def walsh_transform(function_values):
        transformed_values = np.array(function_values) * 2 - 1
        n = len(function_values).bit_length() - 1
        for i in range(n):
            step = 2**(i + 1)
            for j in range(0, len(function_values), step):
                for k in range(2**i):
                    a, b = transformed_values[j + k], transformed_values[j + k + 2**i]
                    transformed_values[j + k], transformed_values[j + k + 2**i] = a + b, a - b
        return transformed_values


    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            bit_independence = sum(
                ((sbox_values[input_value] >> bit1) & 1) ^
                ((sbox_values[input_value] >> bit2) & 1)
                for input_value in range(sbox_size)
            )


            combined_bits = [((sbox_values[input_value] >> bit1) & 1) ^ ((sbox_values[input_value] >> bit2) & 1)
                             for input_value in range(sbox_size)]


            transformed_values = walsh_transform(combined_bits)
            max_walsh_value = np.max(np.abs(transformed_values))


            total_nonlinearity_score += nl

            steps.append({
                "bit_pair": (bit1, bit2),
                "bit_independence": bit_independence,
                "combined_bits": combined_bits,
                "max_walsh_value": max_walsh_value,
                "nonlinearity_value": nl
            })

    bic_nl_score = total_nonlinearity_score / total_bit_pairs

    print("--- Langkah-langkah Perhitungan BIC-NL ---")
    print(f"Total elemen dalam S-box: {sbox_size}")
    for step in steps:
        print(f"\nPasangan bit {step['bit_pair']}:\n  Bit Independence: {step['bit_independence']}\n  Combined Bits: {step['combined_bits']}\n  Max Walsh Value: {step['max_walsh_value']}\n  Nonlinearity Value: {step['nonlinearity_value']:}")

    print(f"\nTotal Nonlinearity Score: {total_nonlinearity_score}")
    print(f"Total Pasangan Bit: {total_bit_pairs}")
    return round(bic_nl_score, 5)

bic_nl_result = calculate_bic_nl(sbox)
print(f"\n--- Hasil Akhir ---")
print(f"Nilai BIC-NL: {bic_nl_result}")

--- Langkah-langkah Perhitungan BIC-NL ---
Total elemen dalam S-box: 256

Pasangan bit (0, 1):
  Bit Independence: 128
  Combined Bits: [0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
  Max Walsh Value: 32
  Nonlinearity Value: 112.0

Pasangan bit (0, 2):
  Bit Independence: 128

# **BIC-SAC**

In [ ]:
def calculate_bic_sac_steps(sbox):
    box_size = len(sbox)
    bit_size = 8
    total_combinations = bit_size * (bit_size - 1) / 2
    cumulative_score = 0

    steps = []


    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            bit_independence = 0
            for input_val in range(box_size):
                for flipped_bit in range(bit_size):
                    new_value = sbox[input_val] ^ sbox[input_val ^ (1 << flipped_bit)]
                    bit1_value = (new_value >> bit1) & 1
                    bit2_value = (new_value >> bit2) & 1
                    bit_independence += bit1_value ^ bit2_value


            normalized_independence = bit_independence / (box_size * bit_size)
            cumulative_score += normalized_independence
            steps.append((bit1, bit2, bit_independence, normalized_independence))

    bic_sac_score = cumulative_score / total_combinations

    return round(bic_sac_score, 5), steps

bic_sac_result, calculation_steps = calculate_bic_sac_steps(sbox)

print(f"--- Langkah-langkah Perhitungan BIC-SAC ---")
print(f"Total elemen dalam S-box: {len(sbox)}")
for step in calculation_steps:
    bit1, bit2, bit_independence, normalized = step
    print(f"Pasangan bit ({bit1}, {bit2}):")
    print(f"  Total bit independence: {bit_independence}")
    print(f"  Normalized independence: {normalized:.5f}")

print(f"\n--- Hasil Akhir ---")
print(f"Nilai BIC-SAC: {bic_sac_result}")

--- Langkah-langkah Perhitungan BIC-SAC ---
Total elemen dalam S-box: 256
Pasangan bit (0, 1):
  Total bit independence: 1004
  Normalized independence: 0.49023
Pasangan bit (0, 2):
  Total bit independence: 1008
  Normalized independence: 0.49219
Pasangan bit (0, 3):
  Total bit independence: 1016
  Normalized independence: 0.49609
Pasangan bit (0, 4):
  Total bit independence: 1052
  Normalized independence: 0.51367
Pasangan bit (0, 5):
  Total bit independence: 1004
  Normalized independence: 0.49023
Pasangan bit (0, 6):
  Total bit independence: 1004
  Normalized independence: 0.49023
Pasangan bit (0, 7):
  Total bit independence: 1028
  Normalized independence: 0.50195
Pasangan bit (1, 2):
  Total bit independence: 1024
  Normalized independence: 0.50000
Pasangan bit (1, 3):
  Total bit independence: 1032
  Normalized independence: 0.50391
Pasangan bit (1, 4):
  Total bit independence: 1028
  Normalized independence: 0.50195
Pasangan bit (1, 5):
  Total bit independence: 1008
  No

# **DAP**

In [ ]:
def calculate_dap(sbox):
    box_size = len(sbox)
    differential_table = [[0] * box_size for _ in range(box_size)]

    for input_diff in range(box_size):
        for input_x in range(box_size):
            output_diff = sbox[input_x] ^ sbox[input_x ^ input_diff]
            differential_table[input_diff][output_diff] += 1

    max_dap = max(
        differential_table[input_diff][output_diff] / box_size
        for input_diff in range(1, box_size)
        for output_diff in range(box_size)
    )


    print("--- Tabel Diferensial ---")
    for i, row in enumerate(differential_table):
        print(f"Input Difference {i}: {row}")

    return round(max_dap, 6)

dap_result = calculate_dap(sbox)

print("\n--- Hasil Akhir DAP ---")
print(f"Nilai DAP: {dap_result}")

--- Tabel Diferensial ---
Input Difference 0: [256, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Input Difference 1: [0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 

# **LAP**

In [ ]:
n = 8
lap = np.zeros((2**n, 2**n), dtype=float)

for input_mask in range(2**n):
    for output_mask in range(2**n):
        count = 0
        for x in range(2**n):
            input_parity = bin(x & input_mask).count('1') % 2
            output_parity = bin(sbox[x] & output_mask).count('1') % 2
            if input_parity == output_parity:
                count += 1
        lap[input_mask][output_mask] = abs(count - (2**(n-1))) / (2**n)

print(lap)

if np.any(lap == 0.0625):
    print("\nLAP menghasilkan nilai 0.0625")
    lap_result = 0.0625
else:
    print("\n LAP tidak menghasilkan nilai 0.0625")

[[0.5       0.        0.        ... 0.        0.        0.       ]
 [0.        0.046875  0.0234375 ... 0.015625  0.0546875 0.0078125]
 [0.        0.015625  0.        ... 0.0234375 0.0390625 0.0234375]
 ...
 [0.        0.0234375 0.03125   ... 0.        0.0546875 0.       ]
 [0.        0.0546875 0.0078125 ... 0.0546875 0.0390625 0.015625 ]
 [0.        0.0546875 0.046875  ... 0.0390625 0.0625    0.0078125]]

LAP menghasilkan nilai 0.0625


# **Kesimpulan**

In [ ]:
print("Nonlinearity (NL):", nl)
print(f"SAC: {final_sac:.5f}")
print("BIC-NL:", bic_nl_result)
print("BIC-SAC:", bic_sac_result)
print("LAP:", lap_result)
print("DAP:", dap_result)

Nonlinearity (NL): 112.0
SAC: 0.50073
BIC-NL: 112.0
BIC-SAC: 0.50237
LAP: 0.0625
DAP: 0.015625


# GUI

## Manual tanpa excel

In [ ]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

sbox = [
    99, 205, 85, 71, 25, 127, 113, 219, 63, 244, 109, 159, 11, 228, 94, 214,
    77, 177, 201, 78, 5, 48, 29, 30, 87, 96, 193, 80, 156, 200, 216, 86,
    116, 143, 10, 14, 54, 169, 148, 68, 49, 75, 171, 157, 92, 114, 188, 194,
    121, 220, 131, 210, 83, 135, 250, 149, 253, 72, 182, 33, 190, 141, 249, 82,
    232, 50, 21, 84, 215, 242, 180, 198, 168, 167, 103, 122, 152, 162, 145, 184,
    43, 237, 119, 183, 7, 12, 125, 55, 252, 206, 235, 160, 140, 133, 179, 192,
    110, 176, 221, 134, 19, 6, 187, 59, 26, 129, 112, 73, 175, 45, 24, 218,
    44, 66, 151, 32, 137, 31, 35, 147, 236, 247, 117, 132, 79, 136, 154, 105,
    199, 101, 203, 52, 57, 4, 153, 197, 88, 76, 202, 174, 233, 62, 208, 91,
    231, 53, 1, 124, 0, 28, 142, 170, 158, 51, 226, 65, 123, 186, 239, 246,
    38, 56, 36, 108, 8, 126, 9, 189, 81, 234, 212, 224, 13, 3, 40, 64,
    172, 74, 181, 118, 39, 227, 130, 89, 245, 166, 16, 61, 106, 196, 211, 107,
    229, 195, 138, 18, 93, 207, 240, 95, 58, 255, 209, 217, 15, 111, 46, 173,
    223, 42, 115, 238, 139, 243, 23, 98, 100, 178, 37, 97, 191, 213, 222, 155,
    165, 2, 146, 204, 120, 241, 163, 128, 22, 90, 60, 185, 67, 34, 27, 248,
    164, 69, 41, 230, 104, 47, 144, 251, 20, 17, 150, 225, 254, 161, 102, 70
]

def hamming_weight(x):
    return bin(x).count('1')

def calculate_sac(sbox, n):
    total_weight = 0
    total_cases = 0

    for i in range(2**n):
        original_output = sbox[i]
        for bit in range(n):
            flipped_input = i ^ (1 << bit)
            flipped_output = sbox[flipped_input]
            diff = original_output ^ flipped_output
            weight = hamming_weight(diff)
            total_weight += weight
            total_cases += n

    return total_weight / total_cases

def calculate_nonlinearity(sbox):
    n = int(np.log2(len(sbox)))
    m = n

    sbox = np.array([list(map(int, f"{val:0{m}b}")) for val in sbox])

    nonlinearity = float('inf')
    for i in range(m):
        boolean_function = sbox[:, i]
        walsh_transform = walsh_hadamard_transform(boolean_function)
        max_correlation = np.max(np.abs(walsh_transform))
        current_nonlinearity = 2**(n-1) - (max_correlation // 2)
        nonlinearity = min(nonlinearity, current_nonlinearity)

    return float(nonlinearity)

def walsh_hadamard_transform(boolean_function):
    n = boolean_function.size
    transformed = 1 - 2 * boolean_function

    for i in range(int(np.log2(n))):
        step = 2**i
        for j in range(0, n, 2*step):
            for k in range(step):
                a = transformed[j + k]
                b = transformed[j + k + step]
                transformed[j + k] = a + b
                transformed[j + k + step] = a - b

    return transformed

def calculate_bic_nl(sbox):
    sbox_size = len(sbox)
    bit_size = 8
    total_bit_pairs = bit_size * (bit_size - 1) / 2
    total_nonlinearity_score = 0

    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            combined_bits = [((sbox[input_value] >> bit1) & 1) ^ ((sbox[input_value] >> bit2) & 1)
                             for input_value in range(sbox_size)]

            transformed_values = walsh_hadamard_transform(np.array(combined_bits))
            max_walsh_value = np.max(np.abs(transformed_values))
            nl = 2**(bit_size - 1) - (max_walsh_value // 2)
            total_nonlinearity_score += nl

    bic_nl_score = total_nonlinearity_score / total_bit_pairs
    return round(bic_nl_score, 5)

def calculate_bic_sac(sbox):
    box_size = len(sbox)
    bit_size = 8
    total_combinations = bit_size * (bit_size - 1) / 2
    cumulative_score = 0

    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            bit_independence = 0
            for input_val in range(box_size):
                for flipped_bit in range(bit_size):
                    new_value = sbox[input_val] ^ sbox[input_val ^ (1 << flipped_bit)]
                    bit1_value = (new_value >> bit1) & 1
                    bit2_value = (new_value >> bit2) & 1
                    bit_independence += bit1_value ^ bit2_value

            normalized_independence = bit_independence / (box_size * bit_size)
            cumulative_score += normalized_independence

    bic_sac_score = cumulative_score / total_combinations
    return round(bic_sac_score, 5)

def calculate_dap(sbox):
    box_size = len(sbox)
    differential_table = [[0] * box_size for _ in range(box_size)]

    for input_diff in range(box_size):
        for input_x in range(box_size):
            output_diff = sbox[input_x] ^ sbox[input_x ^ input_diff]
            differential_table[input_diff][output_diff] += 1

    max_dap = max(
        differential_table[input_diff][output_diff] / box_size
        for input_diff in range(1, box_size)
        for output_diff in range(box_size)
    )

    return round(max_dap, 6)

def calculate_lap(sbox):
    box_size = len(sbox)
    bit_size = int(np.log2(box_size))

    input_masks = np.arange(box_size)
    output_masks = np.arange(box_size)
    sbox_array = np.array(sbox)

    input_parity = np.array([[bin(x & mask).count('1') % 2 for mask in input_masks] for x in range(box_size)])
    output_parity = np.array([[bin(sbox_array[x] & mask).count('1') % 2 for mask in output_masks] for x in range(box_size)])

    correlation_table = np.abs(np.sum(input_parity[:, :, None] == output_parity[:, None, :], axis=0) - (box_size // 2))

    max_lap = np.max(correlation_table[1:, 1:]) / box_size
    return round(max_lap, 6)

sbox_input = widgets.Textarea(
    value=", ".join(map(str, sbox)),
    placeholder="Masukkan S-box sebagai angka dipisahkan koma",
    description="S-box:",
    layout=widgets.Layout(width="100%")
)

operation_select = widgets.Dropdown(
    options=['Nonlinearity (NL)', 'SAC',  'BIC-NL', 'BIC-SAC', 'LAP', 'DAP'],
    value='Nonlinearity (NL)',
    description='Operasi:'
)

run_button = widgets.Button(description="Hitung", button_style="success")
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        try:
            sbox = list(map(int, sbox_input.value.split(',')))
            if len(sbox) != 256:
                print("Error: Panjang S-box harus 256 elemen.")
            else:
                if operation_select.value == 'Nonlinearity (NL)':
                    result = calculate_nonlinearity(sbox)
                    print(f"Hasil Nonlinearity: {result:}")
                elif operation_select.value == 'SAC':
                    result = calculate_sac(sbox, 8)
                    print(f"Hasil SAC: {result:.5f}")
                elif operation_select.value == 'BIC-NL':
                    result = calculate_bic_nl(sbox)
                    print(f"Hasil BIC-NL: {result:}")
                elif operation_select.value == 'BIC-SAC':
                    result = calculate_bic_sac(sbox)
                    print(f"Hasil BIC-SAC: {result:}")
                elif operation_select.value == 'LAP':
                    result = calculate_lap(sbox)
                    print(f"Hasil LAP: {result:}")
                elif operation_select.value == 'DAP':
                    result = calculate_dap(sbox)
                    print(f"Hasil DAP: {result:}")
        except Exception as e:
            print("Error:", e)

run_button.on_click(on_button_click)

display(widgets.VBox([sbox_input, operation_select, run_button, output]))

## Pake excel

In [ ]:
import numpy as np
import ipywidgets as widgets
import pandas as pd
from IPython.display import display, clear_output
import os

def load_sbox_from_excel(file_path):
    try:
        df = pd.read_excel(file_path, header=None)
        sbox = df.values.flatten().tolist()

        if len(sbox) == 256:
            return sbox
        else:
            raise ValueError("File Excel harus mengandung 256 values")
    except Exception as e:
        print(f"File error: {e}")
        return None

file_input = widgets.FileUpload(
    description="Unggah File Excel",
    accept=".xlsx",
    multiple=False
)

def hamming_weight(x):
    return bin(x).count('1')

def calculate_sac(sbox, n):
    total_weight = 0
    total_cases = 0

    for i in range(2**n):
        original_output = sbox[i]
        for bit in range(n):
            flipped_input = i ^ (1 << bit)
            flipped_output = sbox[flipped_input]
            diff = original_output ^ flipped_output
            weight = hamming_weight(diff)
            total_weight += weight
            total_cases += n

    return total_weight / total_cases

def calculate_nonlinearity(sbox):
    n = int(np.log2(len(sbox)))
    m = n

    sbox = np.array([list(map(int, f"{val:0{m}b}")) for val in sbox])

    nonlinearity = float('inf')
    for i in range(m):
        boolean_function = sbox[:, i]
        walsh_transform = walsh_hadamard_transform(boolean_function)
        max_correlation = np.max(np.abs(walsh_transform))
        current_nonlinearity = 2**(n-1) - (max_correlation // 2)
        nonlinearity = min(nonlinearity, current_nonlinearity)

    return float(nonlinearity)

def walsh_hadamard_transform(boolean_function):
    n = boolean_function.size
    transformed = 1 - 2 * boolean_function

    for i in range(int(np.log2(n))):
        step = 2**i
        for j in range(0, n, 2*step):
            for k in range(step):
                a = transformed[j + k]
                b = transformed[j + k + step]
                transformed[j + k] = a + b
                transformed[j + k + step] = a - b

    return transformed

def calculate_bic_nl(sbox):
    sbox_size = len(sbox)
    bit_size = 8
    total_bit_pairs = bit_size * (bit_size - 1) / 2
    total_nonlinearity_score = 0

    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            combined_bits = [((sbox[input_value] >> bit1) & 1) ^ ((sbox[input_value] >> bit2) & 1)
                             for input_value in range(sbox_size)]

            transformed_values = walsh_hadamard_transform(np.array(combined_bits))
            max_walsh_value = np.max(np.abs(transformed_values))
            nl = 2**(bit_size - 1) - (max_walsh_value // 2)
            total_nonlinearity_score += nl

    bic_nl_score = total_nonlinearity_score / total_bit_pairs
    return round(bic_nl_score, 5)

def calculate_bic_sac(sbox):
    box_size = len(sbox)
    bit_size = 8
    total_combinations = bit_size * (bit_size - 1) / 2
    cumulative_score = 0

    for bit1 in range(bit_size):
        for bit2 in range(bit1 + 1, bit_size):
            bit_independence = 0
            for input_val in range(box_size):
                for flipped_bit in range(bit_size):
                    new_value = sbox[input_val] ^ sbox[input_val ^ (1 << flipped_bit)]
                    bit1_value = (new_value >> bit1) & 1
                    bit2_value = (new_value >> bit2) & 1
                    bit_independence += bit1_value ^ bit2_value

            normalized_independence = bit_independence / (box_size * bit_size)
            cumulative_score += normalized_independence

    bic_sac_score = cumulative_score / total_combinations
    return round(bic_sac_score, 5)

def calculate_dap(sbox):
    box_size = len(sbox)
    differential_table = [[0] * box_size for _ in range(box_size)]

    for input_diff in range(box_size):
        for input_x in range(box_size):
            output_diff = sbox[input_x] ^ sbox[input_x ^ input_diff]
            differential_table[input_diff][output_diff] += 1

    max_dap = max(
        differential_table[input_diff][output_diff] / box_size
        for input_diff in range(1, box_size)
        for output_diff in range(box_size)
    )

    return round(max_dap, 6)

def calculate_lap(sbox):
    box_size = len(sbox)
    bit_size = int(np.log2(box_size))

    input_masks = np.arange(box_size)
    output_masks = np.arange(box_size)
    sbox_array = np.array(sbox)

    input_parity = np.array([[bin(x & mask).count('1') % 2 for mask in input_masks] for x in range(box_size)])
    output_parity = np.array([[bin(sbox_array[x] & mask).count('1') % 2 for mask in output_masks] for x in range(box_size)])

    correlation_table = np.abs(np.sum(input_parity[:, :, None] == output_parity[:, None, :], axis=0) - (box_size // 2))

    max_lap = np.max(correlation_table[1:, 1:]) / box_size
    return round(max_lap, 6)

operation_select = widgets.Dropdown(
    options=['Nonlinearity (NL)', 'SAC',  'BIC-NL', 'BIC-SAC', 'LAP', 'DAP'],
    value='Nonlinearity (NL)',
    description='Operasi:'
)

run_button = widgets.Button(description="Hitung", button_style="success")
output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output()
        try:
            if len(file_input.value) == 0:
                print("Error: File Excel belum terunggah.")
                return

            # Get the uploaded file path
            uploaded_file = list(file_input.value.values())[0]
            file_path = uploaded_file['metadata']['name']

            # Save the uploaded file temporarily
            with open(file_path, 'wb') as f:
                f.write(uploaded_file['content'])

            # Load the S-box from the uploaded Excel file
            sbox = load_sbox_from_excel(file_path)
            if sbox is None:
                return

            # Perform the selected operation
            if operation_select.value == 'Nonlinearity (NL)':
                result = calculate_nonlinearity(sbox)
                print(f"Hasil Nonlinearity: {result:}")
            elif operation_select.value == 'SAC':
                result = calculate_sac(sbox, 8)
                print(f"Hasil SAC: {result:.5f}")
            elif operation_select.value == 'BIC-NL':
                result = calculate_bic_nl(sbox)
                print(f"Hasil BIC-NL: {result:}")
            elif operation_select.value == 'BIC-SAC':
                result = calculate_bic_sac(sbox)
                print(f"Hasil BIC-SAC: {result:}")
            elif operation_select.value == 'LAP':
                result = calculate_lap(sbox)
                print(f"Hasil LAP: {result:}")
            elif operation_select.value == 'DAP':
                result = calculate_dap(sbox)
                print(f"Hasil DAP: {result:}")
            os.remove(file_path)

        except Exception as e:
            print("Error:", e)

run_button.on_click(on_button_click)

display(widgets.VBox([file_input, operation_select, run_button, output]))